# Работа с поисковыми движками

**Нужно:**
* выбрать какой-либо новостной ресурс (например http://lenta.ru, http://fontanka.ru, http://gazeta.ru)
* загрузить новости за пару лет
* проиндексировать их с помощью Elasticsearch/Solr/чистая Lucene
* реализовать простой web-интерфес поиска (лучше) или поиск через коммандную строку (хуже)

### Elastichsearch

1. Качаем https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-6.0.1.zip
2. Распаковывам
3. Запускаем `bin\elasticseach`
4. Ставим клиент для Python `python -m pip install -U elasticsearch elasticsearch_dsl`
5. Соединяемся

In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch()

Создаем новый индекс

In [2]:
es.indices.create(index='test-index')

{'acknowledged': True, 'index': 'test-index', 'shards_acknowledged': True}

Добавляем новый документ в индекс

In [3]:
es.index(index='test-index',  doc_type='test',  body={'title': 'Hello', 'content' : 'World hello!'})

{'_id': 'a_kITGABDP7xNZfkjxeJ',
 '_index': 'test-index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'test',
 '_version': 1,
 'result': 'created'}

и ещё один

In [4]:
es.index(index='test-index', doc_type='test', id=2, body={'title': 'Sun', 'content' : 'shine brightly'})

{'_id': '2',
 '_index': 'test-index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'test',
 '_version': 1,
 'result': 'created'}

Теперь ищем

In [7]:
a = es.search(index='test-index', doc_type='test', q='sun')
a

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '2',
    '_index': 'test-index',
    '_score': 0.2876821,
    '_source': {'content': 'shine brightly', 'title': 'Sun'},
    '_type': 'test'}],
  'max_score': 0.2876821,
  'total': 1},
 'timed_out': False,
 'took': 7}

### Выполнение задачи

In [32]:
import os
from tqdm import tqdm_notebook

file_dir = 'no_normilize/'
files = os.listdir(file_dir)
for file in files:
    id = 1
    with open(file_dir + file, 'r') as f, open('indexes_files/' + file, 'w') as outfile:
        for line in f.readlines():
            outfile.write(str(id) + '\t' + line.replace('\n', '') + '\n')
            id = id + 1

In [35]:
import os
import pymorphy2
import re

es.indices.create(index='news')
file_dir = 'no_normilize/'
files = os.listdir(file_dir)
normilizer = pymorphy2.MorphAnalyzer()
for file in files:
    with open(file_dir + file, 'r') as f:
        for line in f.readlines():
            #norm_words = [re.sub('\d+', '', normilizer.normal_forms(x)[0]) for x in re.split('\W+', line) if len(x) > 0]
            es.index(index='news', doc_type='new', body={'content' : line})

In [14]:
a = es.search(index='news-index', doc_type='new', q='парад')

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '5195',
    '_index': 'news-index',
    '_score': 9.099816,
    '_source': {'content': 'в петербург пройти очередной репетиция парад к день вмф по нева днём 23 июль пройти не маленький пятнадцать корабль и катер среди который быть ракетный катер серпухов и димитровград противодиверсионный катер плата 349 малый ракетный корабль гейзер на время репетиция быть развести благовещенский дворцовый троицкий и литейный мост над адмиралтейство пролететь несколько самолёт в тот число дозаправщик и истребитель в петербург пройти очередной репетиция парад к день вмф напомнить парад пройти в петербург и кронштадт 30 июль в немой принять участие корабль балтийский черноморский северный флот каспийский флотилия и авиация тихоокеанский флот принимать парад быть президент следующий репетиция наметить на 28 июль\n'},
    '_type': 'new'},
   {'_id': '1482',
    '_index': 'news-index',
    '_score': 8.283231,
 

In [36]:
a = es.search(index='news-index', doc_type='new', q='парад')
a['hits']['hits'][0]['_source']['content']

'в петербург пройти очередной репетиция парад к день вмф по нева днём 23 июль пройти не маленький пятнадцать корабль и катер среди который быть ракетный катер серпухов и димитровград противодиверсионный катер плата 349 малый ракетный корабль гейзер на время репетиция быть развести благовещенский дворцовый троицкий и литейный мост над адмиралтейство пролететь несколько самолёт в тот число дозаправщик и истребитель в петербург пройти очередной репетиция парад к день вмф напомнить парад пройти в петербург и кронштадт 30 июль в немой принять участие корабль балтийский черноморский северный флот каспийский флотилия и авиация тихоокеанский флот принимать парад быть президент следующий репетиция наметить на 28 июль\n'

In [ ]:
import telebot

token='492410478:AAHINWUA3xjg682Y_d1XpBYkuse4ujIhW60'
bot = telebot.TeleBot(token)

@bot.message_handler(content_types=["text"])
def repeat_all_messages(message): # TODO: текст сообщения нормализовать
    line = message.text
    a = es.search(index='news', doc_type='new', q=line)    
    bot.send_message(message.chat.id, a['hits']['hits'][0]['_source']['content'])
if __name__ == '__main__':
    bot.polling(none_stop=True)

2017-12-12 22:53:13,252 (util.py:64 PollingThread) ERROR - TeleBot: "ApiException occurred, args=('A request to the Telegram API was unsuccessful. The server returned HTTP 409 Conflict. Response body:\n[b\'{"ok":false,"error_code":409,"description":"Conflict: terminated by other long poll or webhook"}\']',)
Traceback (most recent call last):
  File "C:\Users\Veotani\Anaconda3\lib\site-packages\telebot\util.py", line 58, in run
    task(*args, **kwargs)
  File "C:\Users\Veotani\Anaconda3\lib\site-packages\telebot\__init__.py", line 159, in __retrieve_updates
    updates = self.get_updates(offset=(self.last_update_id + 1), timeout=timeout)
  File "C:\Users\Veotani\Anaconda3\lib\site-packages\telebot\__init__.py", line 129, in get_updates
    json_updates = apihelper.get_updates(self.token, offset, limit, timeout, allowed_updates)
  File "C:\Users\Veotani\Anaconda3\lib\site-packages\telebot\apihelper.py", line 175, in get_updates
    return _make_request(token, method_url, params=payload)